<a href="https://colab.research.google.com/github/pratiksha6341/pratiksha6341/blob/main/Copy_of_Instagram_Reach_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'instagram-reach-analysis-case-study:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3404198%2F5929375%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T011201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D261fcbe8e4c59a0aaf07f1593f4cbabd963f369e279daf4b023065659962bbfaa411d59f201c20665032463a9498bd1b776c9cad0c1fe641bc578ff558ae6775fbc8e4b9117a0cbc401fb74a1804af9a3144fbf202a2ad5196e3e8ddeeae373cef8d790855acb645cc7f9cf06456d2a9fe8bcbdcf7b7e9ea286a69406515bf21d5481310daedd2f24d6090c930dcd9a0e9946ae4c8db1f9d0e5270c124160ce9203d152c5ab4e82d1a291edbaf8ab5566d2914060875265a00e8010cd0ad1f13d02bc1c07f0e94cf273292ceda3c8fda2056ac4ed9dd6513e1e78a7b9dd33ac7e89ae5c0798424e65c640846a61685b9df3c6fc0a54e4330e00be8b81eec379c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 11219 bytes downloaded
Downloaded and uncompressed: instagram-reach-analysis-case-study
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/instagram-reach-analysis-case-study/Instagram_data_by_Bhanu.csv


In [ ]:
df = pd.read_csv('/kaggle/input/instagram-reach-analysis-case-study/Instagram_data_by_Bhanu.csv', encoding='latin1')


In [ ]:
df.head(2)

,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags
0,3920,2586,1028,619,56,98,9,5,162,35,2,Here are some of the most important data visua...,#finance #money #business #investing #investme...
1,5394,2727,1838,1174,78,194,7,14,224,48,10,Here are some of the best data science project...,#healthcare #health #covid #data #datascience ...


In [ ]:
! pip install sentence_transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df['caption_embd'] = df['Caption'].apply(lambda x: model.encode(x))
df.head()

,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags,caption_embd
0,3920,2586,1028,619,56,98,9,5,162,35,2,Here are some of the most important data visua...,#finance #money #business #investing #investme...,"[0.09122387, -0.04272804, -0.072459124, 0.0214..."
1,5394,2727,1838,1174,78,194,7,14,224,48,10,Here are some of the best data science project...,#healthcare #health #covid #data #datascience ...,"[-0.044997398, 0.00624404, 0.044310953, -0.029..."
2,4021,2085,1188,0,533,41,11,1,131,62,12,Learn how to train a machine learning model an...,#data #datascience #dataanalysis #dataanalytic...,"[-0.06059351, -0.018961199, -0.020766333, 0.04..."
3,4528,2700,621,932,73,172,10,7,213,23,8,Heres how you can write a Python program to d...,#python #pythonprogramming #pythonprojects #py...,"[0.015730843, 0.047866575, -0.012984105, 0.008..."
4,2518,1704,255,279,37,96,5,4,123,8,0,Plotting annotations while visualizing your da...,#datavisualization #datascience #data #dataana...,"[0.049840014, 0.020660348, -0.076642536, -0.04..."


In [ ]:
df= df.reset_index().rename(columns={'index':'post_id'})
df.head()

,post_id,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags,caption_embd
0,0,3920,2586,1028,619,56,98,9,5,162,35,2,Here are some of the most important data visua...,#finance #money #business #investing #investme...,"[0.09122387, -0.04272804, -0.072459124, 0.0214..."
1,1,5394,2727,1838,1174,78,194,7,14,224,48,10,Here are some of the best data science project...,#healthcare #health #covid #data #datascience ...,"[-0.044997398, 0.00624404, 0.044310953, -0.029..."
2,2,4021,2085,1188,0,533,41,11,1,131,62,12,Learn how to train a machine learning model an...,#data #datascience #dataanalysis #dataanalytic...,"[-0.06059351, -0.018961199, -0.020766333, 0.04..."
3,3,4528,2700,621,932,73,172,10,7,213,23,8,Heres how you can write a Python program to d...,#python #pythonprogramming #pythonprojects #py...,"[0.015730843, 0.047866575, -0.012984105, 0.008..."
4,4,2518,1704,255,279,37,96,5,4,123,8,0,Plotting annotations while visualizing your da...,#datavisualization #datascience #data #dataana...,"[0.049840014, 0.020660348, -0.076642536, -0.04..."


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
embd_arr = np.vstack(df['caption_embd'])
embd_arr

array([[ 0.09122387, -0.04272804, -0.07245912, ...,  0.01991476,
        -0.02285059,  0.0172003 ],
       [-0.0449974 ,  0.00624404,  0.04431095, ...,  0.03802408,
        -0.07262022,  0.03053484],
       [-0.06059351, -0.0189612 , -0.02076633, ...,  0.1303184 ,
         0.05566225, -0.00752302],
       ...,
       [ 0.04858386, -0.08646659, -0.04082508, ...,  0.03436972,
         0.04066313, -0.01645444],
       [-0.03120462, -0.10585386,  0.03789132, ..., -0.00639394,
        -0.00770473,  0.04854013],
       [-0.14428797,  0.00476501, -0.06089444, ...,  0.09543332,
         0.06126888, -0.02384641]], dtype=float32)

In [1]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(embd_arr)

NameError: name 'StandardScaler' is not defined

In [ ]:
kmeans = KMeans(
    init="random",
    n_clusters=3,
    n_init=10,
    max_iter=300,
    random_state=42
)
kmeans.fit(scaled_features)

KMeans(init='random', n_clusters=3, n_init=10, random_state=42)

In [ ]:
label_arr = kmeans.labels_
label_arr

array([0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 2, 1, 2, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 2, 1, 2, 1, 0, 0,
       0, 1, 0, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1], dtype=int32)

In [ ]:
df['cluster_no'] = pd.Series(label_arr)
df.head()

,post_id,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags,caption_embd,cluster_no
0,0,3920,2586,1028,619,56,98,9,5,162,35,2,Here are some of the most important data visua...,#finance #money #business #investing #investme...,"[0.09122387, -0.04272804, -0.072459124, 0.0214...",0
1,1,5394,2727,1838,1174,78,194,7,14,224,48,10,Here are some of the best data science project...,#healthcare #health #covid #data #datascience ...,"[-0.044997398, 0.00624404, 0.044310953, -0.029...",0
2,2,4021,2085,1188,0,533,41,11,1,131,62,12,Learn how to train a machine learning model an...,#data #datascience #dataanalysis #dataanalytic...,"[-0.06059351, -0.018961199, -0.020766333, 0.04...",1
3,3,4528,2700,621,932,73,172,10,7,213,23,8,Heres how you can write a Python program to d...,#python #pythonprogramming #pythonprojects #py...,"[0.015730843, 0.047866575, -0.012984105, 0.008...",1
4,4,2518,1704,255,279,37,96,5,4,123,8,0,Plotting annotations while visualizing your da...,#datavisualization #datascience #data #dataana...,"[0.049840014, 0.020660348, -0.076642536, -0.04...",1


In [ ]:
final_df = df.sort_values(by=['Impressions'], ascending=False)
final_df.head(20)

,post_id,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags,caption_embd,cluster_no
118,118,36919,13473,4176,16444,2547,653,5,26,443,611,228,175 Python Projects with Source Code solved an...,#python #pythonprogramming #pythonprojects #py...,"[-0.14428797, 0.004765013, -0.060894445, -0.03...",1
117,117,32695,11815,3147,17414,170,1095,2,75,549,148,214,Here are some of the best data science certifi...,#datascience #datasciencejobs #datasciencetrai...,"[-0.031204622, -0.105853856, 0.037891325, 0.02...",0
109,109,17713,2449,2141,12389,561,504,3,23,308,70,96,Here are some of the best resources to learn S...,#sql #mysql #datascience #datasciencejobs #dat...,"[0.04806018, -0.12797599, -0.007745143, 0.0519...",0
107,107,17396,1817,10008,5192,251,285,7,7,416,467,260,Here is a list of 100+ Machine Learning Algori...,#machinelearning #machinelearningalgorithms #d...,"[-0.057997964, -0.050237294, -0.021430755, -0....",1
40,40,16062,3144,11817,564,468,252,6,20,416,330,94,280 Machine Learning Projects Solved & Explain...,#data #datascience #dataanalysis #dataanalytic...,"[-0.064429015, -0.015530505, -0.023411358, -0....",1
114,114,13700,5185,3041,5352,77,573,2,38,373,73,80,Here are some of the best data science certifi...,#datascience #datasciencejobs #datasciencetrai...,"[-0.031204622, -0.105853856, 0.037891325, 0.02...",0
112,112,11149,4439,747,5762,53,273,4,13,210,61,58,Python is one of the best programming language...,#python #pythonprogramming #pythoncode #python...,"[0.022777107, -0.011501149, 0.0005641922, -0.0...",1
105,105,11068,2099,2986,5634,122,214,7,8,250,39,34,Here are some of the best data science project...,#data #datascience #dataanalysis #dataanalytic...,"[-0.04503309, -0.033956002, 0.025277607, 0.022...",0
67,67,10933,3152,6610,623,334,225,13,15,301,347,94,170 Python Projects with Source Code solved an...,#python #pythonprogramming #pythonprojects #py...,"[-0.15064454, 0.006005984, -0.06515919, -0.044...",1
54,54,10667,3152,6564,617,187,219,13,15,297,306,74,170 Python Projects with Source Code solved an...,#python #pythonprogramming #pythonprojects #py...,"[-0.15064454, 0.006005984, -0.06515919, -0.044...",1


In [ ]:
final_df[final_df['cluster_no']==2]

,post_id,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags,caption_embd,cluster_no
30,30,6666,1977,4604,48,23,84,9,3,214,57,20,Google Search Analysis is one of the best data...,#data #datascience #dataanalysis #dataanalytic...,"[-0.027938819, 0.0019735831, -0.047270756, -0....",2
15,15,5055,2017,2351,298,108,101,7,11,159,17,6,"In Data Science, Time Series Analysis is a met...",#timeseries #time #statistics #datascience #bi...,"[-0.07146795, 0.009947572, -0.032489013, 0.013...",2
82,82,5055,2017,2351,298,108,101,7,11,159,17,6,"In Data Science, Time Series Analysis is a met...",#timeseries #time #statistics #datascience #bi...,"[-0.07146795, 0.009947572, -0.032489013, 0.013...",2
17,17,3169,1979,707,341,32,106,8,1,121,21,2,"In Data Science, Time Series Analysis is a met...",#timeseries #time #statistics #datascience #bi...,"[-0.08980097, 0.028816616, -0.01115747, -0.004...",2
84,84,3169,1979,707,341,32,106,8,1,121,21,2,"In Data Science, Time Series Analysis is a met...",#timeseries #time #statistics #datascience #bi...,"[-0.08980097, 0.028816616, -0.01115747, -0.004...",2
91,91,2998,1945,794,84,139,42,4,1,126,31,10,Time series analysis means analyzing and findi...,#timeseries #time #statistics #datascience #bi...,"[-0.08160817, 0.03331421, -0.02840927, 0.03488...",2
50,50,2998,1945,794,84,139,42,4,1,126,31,10,Time series analysis means analyzing and findi...,#timeseries #time #statistics #datascience #bi...,"[-0.08160817, 0.03331421, -0.02840927, 0.03488...",2


In [ ]:
print("Cluster 0 count:", len(final_df[final_df['cluster_no']==0]))
print("Cluster 1 count:", len(final_df[final_df['cluster_no']==1]))
print("Cluster 2 count:", len(final_df[final_df['cluster_no']==2]))

Cluster 0 count: 47
Cluster 1 count: 65
Cluster 2 count: 7


# Data Exploration:

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.nunique()

In [ ]:
df.describe().transpose()

# Correlation Analysis:

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

# Print the correlation matrix
print(correlation_matrix)

# Focus on the correlation between "Impressions" and other variables
impressions_correlations = correlation_matrix['Impressions'].drop('Impressions')
print(impressions_correlations)

In [ ]:
df.corr

# Feature Engineering:

In [ ]:
# Derive the "Engagement Rate" feature
df['Engagement Rate'] = (df['Likes'] + df['Comments'] + df['Shares']) / df['Impressions']

# Print the updated DataFrame
print(df.head())

In [ ]:
df.head(10).transpose()

# Exploratory Data Analysis:

In [ ]:
df.columns

In [ ]:
# Analyze trends and patterns in features
# Example 1: Trend of Reach over time
plt.figure(figsize=(12, 6))
sns.lineplot(x=df.index, y=df['Impressions'])
plt.xlabel('Post Index')
plt.ylabel('Reach (Impressions)')
plt.title('Trend of Reach over Time')
plt.show()

# Example 2: Bar chart of Reach from Different Sources
plt.figure(figsize=(10, 6))
sns.barplot(x=['From Home', 'From Hashtags', 'From Explore', 'From Other'],
            y=[df['From Home'].mean(), df['From Hashtags'].mean(), df['From Explore'].mean(), df['From Other'].mean()])
plt.xlabel('Source')
plt.ylabel('Reach')
plt.title('Reach from Different Sources')
plt.show()

# Analyze correlations between features
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Analyze factors such as post type, time of posting, and content themes
# Example 3: Box plot of Reach by Post Type
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['Profile Visits'], y=df['Impressions'])
plt.xlabel('Profile Visits')
plt.ylabel('Reach (Impressions)')
plt.title('Reach by Post Type')
plt.show()

# Example 4: Word cloud of the most used hashtags
from wordcloud import WordCloud

hashtags = ' '.join(df['Hashtags'].dropna().tolist())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(hashtags)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Used Hashtags')
plt.show()

In [ ]:
df.index

In [ ]:
df.columns

# Predictive Modeling:

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the Instagram data into a DataFrame
df = pd.read_csv('/kaggle/input/instagram-reach-analysis-case-study/Instagram_data_by_Bhanu.csv', encoding='latin1')

# Select the relevant features for prediction
X = df[['From Home', 'From Hashtags', 'From Explore', 'From Other', 'Saves', 'Comments', 'Shares', 'Likes', 'Profile Visits', 'Follows']]
y = df['Impressions']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Select and train the machine learning model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's performance using appropriate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Use the trained model to predict the reach for new or future posts
new_data = pd.DataFrame({
    'From Home': [10],  # Example value, adjust according to your needs
    'From Hashtags': [5],  # Example value, adjust according to your needs
    'From Explore': [2],  # Example value, adjust according to your needs
    'From Other': [3],  # Example value, adjust according to your needs
    'Saves': [50],  # Example value, adjust according to your needs
    'Comments': [20],  # Example value, adjust according to your needs
    'Shares': [10],  # Example value, adjust according to your needs
    'Likes': [100],  # Example value, adjust according to your needs
    'Profile Visits': [30],  # Example value, adjust according to your needs
    'Follows': [5]  # Example value, adjust according to your needs
})

predicted_reach = model.predict(new_data)
print("Predicted Reach:", predicted_reach)


# Interpretation and Recommendations:

In [ ]:

from sklearn.linear_model import LinearRegression

# Load the Instagram data into a DataFrame
df = pd.read_csv('/kaggle/input/instagram-reach-analysis-case-study/Instagram_data_by_Bhanu.csv', encoding='latin1')

# Select the relevant features for prediction
X = df[['From Home', 'From Hashtags', 'From Explore', 'From Other', 'Saves', 'Comments', 'Shares', 'Likes', 'Profile Visits', 'Follows']]
y = df['Impressions']

# Train the machine learning model
model = LinearRegression()
model.fit(X, y)

# Interpret the coefficients or feature importances
coefficients = model.coef_
feature_importances = abs(coefficients) / abs(coefficients).sum()

# Identify the key factors that significantly influence reach
key_factors = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': coefficients,
    'Feature Importance': feature_importances
})
key_factors = key_factors.sort_values(by='Coefficient', ascending=False)

# Print the key factors and their impact on reach
print(key_factors)

# Recommendations to optimize reach

# 1. Optimize the use of hashtags:
# Based on the impact on reach from hashtags, analyze the hashtags used in successful posts.
# Identify popular and relevant hashtags in the niche and incorporate them strategically.

# 2. Encourage engagement:
# Focus on generating saves, comments, and shares as they positively influence reach.
# Create compelling and shareable content that encourages interaction and discussion among the audience.

# 3. Analyze post types and content themes:
# Explore the impact of different post types (photos, videos) on reach.
# Analyze the performance of various content themes or topics to identify what resonates best with the audience.

# 4. Understand reach from different sources:
# Analyze the reach from different sources such as home, explore, and others.
# Determine the distribution of audience engagement and consider adjusting strategies to target specific sources more effectively.



In [ ]:
df.to_csv(' Instagram_Reach_Analysis_data.csv', index=True)